In [44]:
params = {"Data":'KaggleDR',
        "data_name":None,
        "style":None,
        "model": "KaggleDR",
        "file_path":r"C:\KaggleDR",
        "pickle_path":r"C:\pickles\KaggleDR",
        "model_path":{"Nat":r"C:\models\Epochs_50_Nat.json","KaggleDR":r"C:\models\Epochs_50_kaggleDR.json","Chest":r"C:\models\Epochs_50_Chest.json", "CatDog":r"C:\models\Epochs_40_CatDog.json" },
        "RandomSeed":2,
        "doc_path":r"C:\Users\Flori\Documents\GitHub\t",
        'img_size_x':224,
        'img_size_y':224,
        'norm':False,
        'color':True, 
        'pretrain':None, 
        "equal_data":False, 
        "shuffle":True, 
        "epochs":50, 
        "val_size":5000,
        "test_size":5000, 
        "Batch_size":32,
        "stop":'yes'
        }

In [7]:
params = {"Data":'Nat',
        "style":'none',
        "model":'Nat',
        'file_path':r"C:\natural_images",
        'pickle_path':r"C:\pickles\Nat",
        "model_path":{"Nat":r"C:\models\Epochs_50_Nat.json","KaggleDR":r"C:\models\Epochs_50_kaggleDR.json","Chest":r"C:\models\Epochs_50_Chest.json", "CatDog":r"C:\models\Epochs_40_CatDog.json" },
        "RandomSeed":2,
        "doc_path":r"C:\Users\Flori\Documents\GitHub\t",
        'img_size_x':224,
        'img_size_y':224,
        'norm':False,
        'color':True, 
        'pretrain':None, 
        "equal_data":False, 
        "shuffle":True, 
        "epochs":1, 
        "val_size":500,
        "test_size":1000, 
        "Batch_size":32,
        "stop":'yes'
        }

In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import tensorflow as tf
from AADatasets import *
from AAPreTrain import *
from AATransferLearn import *
from AAlogic import *
from LabnotesDoc import *



config_desktop()


x_test,y_test,x,y = val_split(x,y, params["test_size"])
x_val,y_val,x,y = val_split(x,y, params["val_size"])

model = make_model(x, y, params)
# weights = determen_weights(y)
H, score, model = train_model(model,x,y,x_val,y_val,x_test,y_test, params)

# if params["style"] == 'FT':
#     model = make_model(x, y, params)
#     weights = determen_weights(y)
#     H, score, model = train_model(model,x,y,x_val,y_val,x_test,y_test, params, weights)
#     predictions = get_feature_vector(model, x, layer = 'fc2')
#     predictions_test = get_feature_vector(model, x_test, layer = 'fc2')
#     score = auc_svm(predictions,y,predictions_test,y_test, plot = False)
#     results = {'score':score,"acc_epoch":H.history['acc'],"val_acc_epoch":H.history['val_acc'],"loss_epoch":H.history['loss'],"vall_loss_epoch":H.history['val_loss']}

# elif params["style"] =='SVM':
#     predictions = x.flatten().reshape(len(x), 224*224)
#     predictions_test = x_test.flatten().reshape(len(x_test), 224*224)
#     score = auc_svm(predictions,y,predictions_test,y_test, plot = False)
#     results = {'score':score,'data_name':params["data_name"],'method':params["model"],'style':params["style"]}
#     H=None
path = r'C:\models\NEW'
#save model to JSON
model_json = model.to_json()
# with open(f"{params['model_path']}{params['epochs']}_{params['Data']}.json", "w") as json_file:
#     json_file.write(model_json)
with open(f"{path}ModelChange_{params['model']}.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights(f"{params['model_path']}{params['epochs']}_{params['Data']}_Weights.h5")
# print("Saved model to disk")
model.save_weights(f"{path}ModelChange_{params['model']}_Weights.h5")
print("Saved model to disk")

results = {'score':score,"acc_epoch":H.history['acc'],"val_acc_epoch":H.history['val_acc'],"loss_epoch":H.history['loss'],"vall_loss_epoch":H.history['val_loss']}

doc(params,results,H,params["doc_path"])

In [3]:
def loading(size,i,start, name):
    stop = time.time()
    part = int(((i+1)/size)*20)
    loading_bar = part*'-'+(20-part)*' '
    print(f"{name}: {i+1}/{size}: [{loading_bar[0:10]}{part*5}%{loading_bar[10:20]}] elapsed time: {int(stop-start)}",end='\r')

def more_data(x,y,r):
    x_new = x
    y_new = y
    start = time.time()
    for i in range(0,r):
        loading(r,i,start, "Data generator")
        datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, fill_mode = "nearest")
        img=datagen.random_transform(x[i])
        y_new = np.concatenate((y_new, np.array(y[i]).reshape(1,y.shape[1])))
        x_new = np.concatenate((x_new, np.array(img).reshape(1,img.shape[0], img.shape[1],img.shape[2])))
    print("\n")
    return x_new,y_new

def equal_data(x,y):
    d = count_classes(y)
    l = list()
    for key in d.keys():
        l.append(d[key])
    limit = max(l)

    zip_melanoom = zip(x,y)
    zippy = list(zip_melanoom)
    random.shuffle(zippy)
    x,y = zip(*zippy)
    x = np.array(x)
    y = np.array(y)
    
    bad = list()
    datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, fill_mode = "nearest")

    x_new = x
    y_new = y
    start = time.time()
    for i in range(0,len(y)):
        loading(len(y),i,start, "Data generator")
        
        if not d[list(y[i]).index(1)] < limit:
            bad.append(list(y[i]))
            continue
        if list(y[i]).index(1) in bad:
            continue
        img=datagen.random_transform(x[i])
        y_new = np.concatenate((y_new, np.array(y[i]).reshape(1,y.shape[1])))
        x_new = np.concatenate((x_new, np.array(img).reshape(1,img.shape[0], img.shape[1],img.shape[2])))
        d = count_classes(y_new)
        if not d[list(y[i]).index(1)] < limit:
            bad.append(list(y[i]).index(1))
    print("\n")
    return x_new,y_new

In [4]:
x,y = get_data(params)

Try to import pickle
succeed to import pickle
 unzip


In [6]:
def equal_data_min(x,y):
    d = count_classes(y)
    if type(d) == dict:
        l = list()
        for key in d.keys():
            l.append(d[key])
        limit = min(l)
    else:
        limit = min(d)

    back_to_num = list()
    for i in list(y):
        back_to_num.append(list(i).index(1))
    
    X = list()
    Y = list()
    d = dict()
    for n,img in zip(back_to_num,zip(x,y)):
        if n in d:
            if d[n] >= limit:
                continue
            d[n] += 1
            X.append(img[0])
            Y.append(img[1])
        else:
            d[n] = 1
            X.append(img[0])
            Y.append(img[1])
    return np.array(X),np.array(Y)

In [7]:
x2,y2 = equal_data_min(x,y)

In [11]:
y[0]

array([1, 0, 0, 0, 0])

In [27]:
def data_gen(x,y):
    datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, fill_mode = "nearest")
    start = time.time()
    x_new = x[1000:]
    y_new = y[1000:]
    for i in range(0,len(y)):
        if list(y[i]).index(1) == 0:
            continue
        loading(len(y),i,start, "Data generator")
        img=datagen.random_transform(x[i])
        y_new = np.concatenate((y_new, np.array(y[i]).reshape(1,y.shape[1])))
        x_new = np.concatenate((x_new, np.array(img).reshape(1,img.shape[0], img.shape[1],img.shape[2])))
        
    print("\n")
    return x_new,y_new

In [28]:
x3,y3 = data_gen(x2,y2)

Data generator: 3540/3540: [----------100%----------] elapsed time: 1532



In [38]:
x =0 
x3= 0
x2 = 0

In [35]:
y_new = np.concatenate((y,y3))
x_new = np.concatenate((x,x3))

In [42]:
def val_split(x,y, val_size):
    return x[-val_size:] ,y[-val_size:] ,x[:-val_size] ,y[:-val_size]

In [43]:
x_test,y_test,x,y = val_split(x_new,y_new, params["test_size"])
x_val,y_val,x,y = val_split(x,y, params["val_size"])

In [47]:
count_classes(y)

{0: 22416, 2: 4591, 4: 624, 1: 2107, 3: 760}

In [48]:
weights

{0: 1.6534562211981567,
 1: 9.75936,
 2: 6.116726835138388,
 3: 15.04588061174149,
 4: 15.925848563968668}

In [40]:
model = make_model(x, y, params)
weights = determen_weights(y)

C:\models\Epochs_50_kaggleDR.json
Loaded model from disk
MODEL SUMMARY:
<tensorflow.python.keras._impl.keras.engine.topology.InputLayer object at 0x00000169E7FEAE10> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000168917C5128> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000169E7FEAC18> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x00000169E7FEAA90> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000169E7FEA940> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000169E7FEA4E0> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x00000169487B4128> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000169487B41D0> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000169487B4320> True
<tensorflow.python.keras._imp

In [49]:
H, score, model = train_model(model,x,y,x_val,y_val,x_test,y_test, params,weights)

Train on 30498 samples, validate on 5000 samples
Epoch 1/50
30498/30498 [==============================]30498/30498 [==============================] - 636s 21ms/step - loss: 5.3279 - acc: 0.6468 - val_loss: 1.4136 - val_acc: 0.5588

Epoch 2/50
30498/30498 [==============================]30498/30498 [==============================] - 632s 21ms/step - loss: 5.0717 - acc: 0.6496 - val_loss: 1.3407 - val_acc: 0.6060

Epoch 3/50
30498/30498 [==============================]30498/30498 [==============================] - 632s 21ms/step - loss: 4.8495 - acc: 0.7140 - val_loss: 1.2709 - val_acc: 0.6588

Epoch 4/50
30498/30498 [==============================]30498/30498 [==============================] - 633s 21ms/step - loss: 4.6391 - acc: 0.7592 - val_loss: 1.2001 - val_acc: 0.7160

Epoch 5/50
30498/30498 [==============================]30498/30498 [==============================] - 633s 21ms/step - loss: 4.4354 - acc: 0.7886 - val_loss: 1.1537 - val_acc: 0.7032

Epoch 6/50
30498/30498 [=======

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
model_json = model.to_json()
with open("C:\models\Epochs50_kaggleDR.json", "w") as json_file:
    json_file.write(model_json)

## Serialize weights to HDF5 ##
model.save_weights("C:\models\Epochs50_kaggleDR_Weights.h5")
print("Saved model to disk")

In [16]:
y_proba = model.predict(x)


In [20]:
y_classes = y_proba.argmax(axis=-1)

In [22]:
from sklearn.metrics import confusion_matrix

back_to_num = list()
list_classes = list(y)
for i in list_classes:
    back_to_num.append(list(i).index(1))

y_pred = []
for c in y_proba:
    y_pred.append(list(c).index(max(c)))
    
cnf_matrix = confusion_matrix(back_to_num, y_classes)

In [23]:
cnf_matrix

array([[  0,   0,   0,   3, 551,   0,   9,  10],
       [  2,  10,   0,   5,   1,   0, 733,   4],
       [  5,  14,   0,   3,  14,   4, 597,  55],
       [  3,  17,   0,   4,  12,   5, 127, 390],
       [  1, 605,   0,  10,   8,   5,  22,  20],
       [743,   0,   0,   0,   3,   0,   1,   0],
       [  0,   8,   0, 603,   1,   0,   7,   0],
       [  1, 763,   0,   0,   0,   0,  17,   3]], dtype=int64)

In [18]:
cnf_matrix

array([[  0,   0,   0,   3, 551,   0,   9,  10],
       [  2,  10,   0,   5,   1,   0, 733,   4],
       [  5,  14,   0,   3,  14,   4, 597,  55],
       [  3,  17,   0,   4,  12,   5, 127, 390],
       [  1, 605,   0,  10,   8,   5,  22,  20],
       [743,   0,   0,   0,   3,   0,   1,   0],
       [  0,   8,   0, 603,   1,   0,   7,   0],
       [  1, 763,   0,   0,   0,   0,  17,   3]], dtype=int64)

In [24]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, model.predict(x_test))

In [25]:
score

0.43843318153651234